#### Imports 

In [1]:
from time import gmtime, strftime
from utils import InferenceSpec
import sagemaker
import logging
import boto3
import json

#### Setup

In [2]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [3]:
region = sagemaker.Session().boto_region_name
boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client('sagemaker')
logger.info(f'Region = {region}')

Region = us-east-1


#### List all model package groups

In [4]:
sagemaker_client.list_model_package_groups()['ModelPackageGroupSummaryList']

[{'ModelPackageGroupName': 'BERT-Email-Classifier',
  'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/bert-email-classifier',
  'ModelPackageGroupDescription': 'BERT-based multi-class classifier for email threat detection',
  'CreationTime': datetime.datetime(2021, 8, 6, 20, 29, 54, 651000, tzinfo=tzlocal()),
  'ModelPackageGroupStatus': 'Completed'},
 {'ModelPackageGroupName': 'email-spam-classification',
  'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/email-spam-classification',
  'ModelPackageGroupDescription': 'email spam',
  'CreationTime': datetime.datetime(2021, 8, 5, 17, 1, 6, 613000, tzinfo=tzlocal()),
  'ModelPackageGroupStatus': 'Completed'},
 {'ModelPackageGroupName': 'project-1-ci-cd-p-wphdcrsekdft',
  'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/project-1-ci-cd-p-wphdcrsekdft',
  'CreationTime': datetime.datetime(2021, 7, 26, 13, 20, 29, 726000, tzinfo=tz

### Create a Model Package Group

In [5]:
model_package_group_name = 'BERT-Email-Classifier'
config = {'ModelPackageGroupName': model_package_group_name, 
          'ModelPackageGroupDescription': 'BERT-based multi-class classifier for email threat detection'
         }

In [11]:
sagemaker_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']

[]

In [12]:
if not sagemaker_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']:
    response = sagemaker_client.create_model_package_group(**config)
    logger.info(f'Model Package Group: {model_package_group_name} created!')
    logger.info(f'Response: {response}')
else:
    logger.warning('Model Package Group already exists!')

Model Package Group: BERT-Email-Classifier created!
Response: {'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/bert-email-classifier', 'ResponseMetadata': {'RequestId': 'da9caf1f-0126-4e55-bd32-d1cab032855a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'da9caf1f-0126-4e55-bd32-d1cab032855a', 'content-type': 'application/x-amz-json-1.1', 'content-length': '109', 'date': 'Sun, 08 Aug 2021 03:53:22 GMT'}, 'RetryAttempts': 0}}


#### List Model Packages 
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.list_model_packages

In [13]:
response = sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)
response

{'ModelPackageSummaryList': [],
 'ResponseMetadata': {'RequestId': '47d97220-0857-4c21-8d42-44fdc0845506',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '47d97220-0857-4c21-8d42-44fdc0845506',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '30',
   'date': 'Sun, 08 Aug 2021 03:53:27 GMT'},
  'RetryAttempts': 0}}

#### Describe a Model Package Group

In [14]:
sagemaker_client.describe_model_package_group(ModelPackageGroupName=model_package_group_name)

{'ModelPackageGroupName': 'BERT-Email-Classifier',
 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/bert-email-classifier',
 'ModelPackageGroupDescription': 'BERT-based multi-class classifier for email threat detection',
 'CreationTime': datetime.datetime(2021, 8, 8, 3, 53, 23, 430000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:892313895307:user-profile/d-dowart1jabkf/ts-zd-e2e',
  'UserProfileName': 'ts-zd-e2e',
  'DomainId': 'd-dowart1jabkf'},
 'ModelPackageGroupStatus': 'Completed',
 'ResponseMetadata': {'RequestId': '1e5a5fa3-e291-412b-976c-663b088d7707',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1e5a5fa3-e291-412b-976c-663b088d7707',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '490',
   'date': 'Sun, 08 Aug 2021 03:53:28 GMT'},
  'RetryAttempts': 0}}

### How do I register a trained BERT model to the model registry 

In [15]:
from sagemaker import image_uris
ecr_image = image_uris.retrieve(framework='huggingface',region='us-east-1',version='4.6.1',image_scope='inference',
                    base_framework_version='tensorflow2.4.1', py_version='py37', container_version='ubuntu18.04', 
                               instance_type='ml.m5.4xlarge')
ecr_image

'763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04'

1. Define the inference spec 

In [16]:
# ecr_image = '763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-tensorflow-training:2.4.1-transformers4.6.1-gpu-py37-cu110-ubuntu18.04'
inference_spec = InferenceSpec().get_dict(ecr_image=ecr_image, 
                                                    supports_gpu=True,  
                                                    supported_content_types=['text/csv'], 
                                                    supported_mime_types=['text/csv'])

In [17]:
inference_spec["InferenceSpecification"]["Containers"][0]["ModelDataUrl"] = 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'
inference_spec

{'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04',
    'ModelDataUrl': 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   'ml.c4.8xlarge',
   'ml.c5.xlarge',
   'ml.c5.2xlarge',
   'ml.c5.4xlarge',
   'ml.c5.9xlarge',
   'ml.c5.18xlarge',
   'ml.p2.xlarge',
   'ml.p2.8xlarge',
   'ml.p2.16xlarge',
   'ml.p3.2xlarge',
   'ml.p3.8xlarge',
   'ml.p3.16xlarge'],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xl

**Note:** The ModelDataUrl can also be obtained from a previously run training job.

2. Define/load model metrics 

In [21]:
with open('./metrics.json', 'rb') as f:
    model_metrics = json.load(f)

In [22]:
model_metrics

{'binary_classification_metrics': {'accuracy': {'value': 0.7768782067298889,
   'standard_deviation': 'NaN'},
  'f2': {'value': 0.5370374250411987, 'standard_deviation': 'NaN'}}}

In [23]:
current_timestamp = strftime('%m-%d-%H-%M', gmtime())

Let's push this metrics file to S3

In [24]:
!aws s3 cp metrics.json s3://sagemaker-us-east-1-892313895307/metrics/{current_timestamp}/metrics.json

upload: ./metrics.json to s3://sagemaker-us-east-1-892313895307/metrics/08-08-03-55/metrics.json


In [25]:
model_metrics = {
    "ModelQuality": {
        "Statistics": {
            "ContentType": "application/json",
            "S3Uri": f"s3://sagemaker-us-east-1-892313895307/metrics/{current_timestamp}/metrics.json",
        }
    }
}

3. Create model package config (dictionary)

In [26]:
model_package_input_dict = {"ModelPackageGroupName": model_package_group_name,
                            "ModelPackageDescription": "BERT classifier", 
                            "ModelApprovalStatus": "PendingManualApproval", 
                            "ModelMetrics": model_metrics}

model_package_input_dict.update(inference_spec)
model_package_input_dict

{'ModelPackageGroupName': 'BERT-Email-Classifier',
 'ModelPackageDescription': 'BERT classifier',
 'ModelApprovalStatus': 'PendingManualApproval',
 'ModelMetrics': {'ModelQuality': {'Statistics': {'ContentType': 'application/json',
    'S3Uri': 's3://sagemaker-us-east-1-892313895307/metrics/08-08-03-55/metrics.json'}}},
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04',
    'ModelDataUrl': 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   'ml.c4.8xlarge',
   'ml.c5.xlarge',
   'ml.c5.2xlarge',
   'ml.c5.4xlarge',
   'ml.c5.9xlar

In [27]:

meta = {'MetadataProperties': {'GeneratedBy': 'NLP-Pipeline'}}

model_package_input_dict.update(meta)

In [28]:
model_package_input_dict

{'ModelPackageGroupName': 'BERT-Email-Classifier',
 'ModelPackageDescription': 'BERT classifier',
 'ModelApprovalStatus': 'PendingManualApproval',
 'ModelMetrics': {'ModelQuality': {'Statistics': {'ContentType': 'application/json',
    'S3Uri': 's3://sagemaker-us-east-1-892313895307/metrics/08-08-03-55/metrics.json'}}},
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04',
    'ModelDataUrl': 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   'ml.c4.8xlarge',
   'ml.c5.xlarge',
   'ml.c5.2xlarge',
   'ml.c5.4xlarge',
   'ml.c5.9xlar

4. Create the Model Package 

In [29]:
response = sagemaker_client.create_model_package(**model_package_input_dict)
response

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/1',
 'ResponseMetadata': {'RequestId': 'a0e28ce4-cd83-4994-aea2-ced1f2ffaf02',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a0e28ce4-cd83-4994-aea2-ced1f2ffaf02',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Sun, 08 Aug 2021 03:56:31 GMT'},
  'RetryAttempts': 0}}

In [30]:
arn = response["ModelPackageArn"]
model_package_info = sagemaker_client.describe_model_package(ModelPackageName=arn)
model_package_info

{'ModelPackageGroupName': 'BERT-Email-Classifier',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/1',
 'ModelPackageDescription': 'BERT classifier',
 'CreationTime': datetime.datetime(2021, 8, 8, 3, 56, 31, 360000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04',
    'ImageDigest': 'sha256:8bf391a8d10e8e7f5d26223625e8ab97276fd7dfa2a41075ddd84418055ecbb8',
    'ModelDataUrl': 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   'ml.c4.

In [31]:
status = model_package_info["ModelPackageStatus"]

while status not in ["Completed", "Failed"]:
    time.sleep(5)
    info = sagemaker_client.describe_model_package(ModelPackageName=arn)
    status = info["ModelPackageStatus"]
    logger.info(f"model package status: {status}")
logger.info(f"model package status: {status}")

model package status: Completed


#### Describe a Model Package 

In [32]:
arn = model_package_info['ModelPackageArn']
sagemaker_client.describe_model_package(ModelPackageName=arn)

{'ModelPackageGroupName': 'BERT-Email-Classifier',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/1',
 'ModelPackageDescription': 'BERT classifier',
 'CreationTime': datetime.datetime(2021, 8, 8, 3, 56, 31, 360000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04',
    'ImageDigest': 'sha256:8bf391a8d10e8e7f5d26223625e8ab97276fd7dfa2a41075ddd84418055ecbb8',
    'ModelDataUrl': 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   'ml.c4.

### Update status of a Model Package 

In [33]:
model_package_arn = 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/1'

In [34]:
model_package_update_input_dict = {"ModelPackageArn" : model_package_arn, 
                                   "ModelApprovalStatus" : "Approved"}
model_package_update_response = sagemaker_client.update_model_package(**model_package_update_input_dict)
model_package_update_response

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/1',
 'ResponseMetadata': {'RequestId': '737bb010-b958-4bcf-8d30-2709f1c75e0e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '737bb010-b958-4bcf-8d30-2709f1c75e0e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Sun, 08 Aug 2021 04:00:41 GMT'},
  'RetryAttempts': 0}}

### Deletion

#### Delete all Model Packages in a Model Package Group

List all standalone model packages (ones that are not affiliated to any Model Package Group).

In [6]:
response = sagemaker_client.list_model_packages()
response

{'ModelPackageSummaryList': [],
 'ResponseMetadata': {'RequestId': 'f020336e-1a0c-45d5-9027-7f5b1e59b9cb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f020336e-1a0c-45d5-9027-7f5b1e59b9cb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '30',
   'date': 'Sun, 08 Aug 2021 03:53:07 GMT'},
  'RetryAttempts': 0}}

Delete all standalone model packages 

In [7]:
for model_package in response['ModelPackageSummaryList']:
    logger.info(f'Model Pacakge: {model_package}')
    # package_name = model_package['ModelPackageName']
    package_arn = model_package['ModelPackageArn']
    response = sagemaker_client.delete_model_package(ModelPackageName=package_arn)
    logger.info(f'Response: {response}')

List model packages that are affiliated to our Model Package Group

In [8]:
response = sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)
response

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'BERT-Email-Classifier',
   'ModelPackageVersion': 3,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/3',
   'ModelPackageDescription': 'BERT classifier',
   'CreationTime': datetime.datetime(2021, 8, 6, 21, 50, 44, 482000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'Rejected'},
  {'ModelPackageGroupName': 'BERT-Email-Classifier',
   'ModelPackageVersion': 2,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/2',
   'ModelPackageDescription': 'BERT classifier',
   'CreationTime': datetime.datetime(2021, 8, 6, 20, 44, 35, 589000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'Approved'},
  {'ModelPackageGroupName': 'BERT-Email-Classifier',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-cl

In [9]:
for model_package in response['ModelPackageSummaryList']:
    logger.info(f'Model Package: {model_package}')
    # package_name = model_package['ModelPackageName']
    package_arn = model_package['ModelPackageArn']
    response = sagemaker_client.delete_model_package(ModelPackageName=package_arn)
    logger.info(f'Response: {response}')

Model Package: {'ModelPackageGroupName': 'BERT-Email-Classifier', 'ModelPackageVersion': 3, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/3', 'ModelPackageDescription': 'BERT classifier', 'CreationTime': datetime.datetime(2021, 8, 6, 21, 50, 44, 482000, tzinfo=tzlocal()), 'ModelPackageStatus': 'Completed', 'ModelApprovalStatus': 'Rejected'}
Response: {'ResponseMetadata': {'RequestId': '3431415c-5900-4acf-a13e-15189469cd2d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3431415c-5900-4acf-a13e-15189469cd2d', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Sun, 08 Aug 2021 03:53:12 GMT'}, 'RetryAttempts': 0}}
Model Package: {'ModelPackageGroupName': 'BERT-Email-Classifier', 'ModelPackageVersion': 2, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/2', 'ModelPackageDescription': 'BERT classifier', 'CreationTime': datetime.datetime(2021, 8, 6, 20, 4

#### Delete a Model Package Group if needed

**Note:** We need to delete all Model Packages inside the Model Package Group first to delete the package group.

In [10]:
sagemaker_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)

{'ResponseMetadata': {'RequestId': '86e42e25-ce94-402e-9b5f-ff2d907688ac',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '86e42e25-ce94-402e-9b5f-ff2d907688ac',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 08 Aug 2021 03:53:19 GMT'},
  'RetryAttempts': 0}}

#### How do I recreate the BERT model from the model registry and use it to create a real-time inference endpoint

#### How do I create an endpoint using model artifacts in S3 

#### How do I create an endpoint via Model Registry